In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
import pandas as pd



path = os.listdir('/kaggle/input/reduced-mri-scans-data/dataset/train-data')
classes = {'zero':0, 'one':1}

import cv2
X = []
Y = []
for cls in classes:
    pth = '/kaggle/input/reduced-mri-scans-data/dataset/train-data/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j, 0)
        img = cv2.resize(img, (200,200))
        X.append(img)
        Y.append(classes[cls])

X = np.array(X)
Y = np.array(Y)

X_updated = X.reshape(len(X), -1)
print(len(X))
print(len(Y))

np.unique(Y)

pd.Series(Y).value_counts()

X.shape, X_updated.shape

X_updated = X.reshape(len(X), -1)
X_updated.shape

xtrain, xtest, ytrain, ytest = train_test_split(X_updated, Y, random_state=10,
                                               test_size=.30)

xtrain.shape, xtest.shape

print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())
xtrain = xtrain/255
xtest = xtest/255
print(xtrain.max(), xtrain.min())
print(xtest.max(), xtest.min())


2024-05-10 14:51:54.649687: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 14:51:54.649806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 14:51:54.799237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


12952
12952
255 0
255 0
1.0 0.0
1.0 0.0


In [2]:
from tensorflow.keras.applications import VGG19
from sklearn.metrics import precision_score, recall_score, f1_score

# Load pre-trained VGG19 model without top layers
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze the layers in VGG19 so they are not trainable
for layer in vgg_model.layers:
    layer.trainable = False

# Build the new model on top of VGG19
model = Sequential([
    vgg_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape X data to fit VGG19 input shape
X_reshaped = np.stack((X,) * 3, axis=-1)  # Convert grayscale images to 3-channel images
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, Y, test_size=0.3, random_state=10)


In [4]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


2024-05-10 14:53:36.788699: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.90083, expected 3.24734
2024-05-10 14:53:36.788748: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.41248, expected 3.75899
2024-05-10 14:53:36.788757: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 5.15283, expected 4.49933
2024-05-10 14:53:36.788765: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 5.47684, expected 4.82335
2024-05-10 14:53:36.788773: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.38459, expected 4.7311
2024-05-10 14:53:36.788780: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 4.2626, expected 3.6091
2024-05-10 14:53:36.788788: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 4.75165, expected 4.09815
2024-05-10 14:53:36.788795: E external/local_xla/xla/servi

  2/227 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - accuracy: 0.4375 - loss: 79.3589   

I0000 00:00:1715352830.007965     101 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


226/227 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5723 - loss: 23.1131

2024-05-10 14:54:05.444094: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.79148, expected 2.34842
2024-05-10 14:54:05.444147: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 31: 3.23739, expected 2.79432
2024-05-10 14:54:05.444157: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 67: 2.84633, expected 2.40327
2024-05-10 14:54:05.444164: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 68: 3.26482, expected 2.82175
2024-05-10 14:54:05.444172: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 74: 2.85088, expected 2.40782
2024-05-10 14:54:05.444180: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 75: 3.01775, expected 2.57468
2024-05-10 14:54:05.444188: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 78: 3.39053, expected 2.94747
2024-05-10 14:54:05.444195: E external/local_xla/

227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.5725 - loss: 23.0380

2024-05-10 14:54:19.933178: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.26091, expected 2.7181
2024-05-10 14:54:19.933241: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.38745, expected 3.84463
2024-05-10 14:54:19.933251: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 3.9685, expected 3.42569
2024-05-10 14:54:19.933258: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.54911, expected 3.0063
2024-05-10 14:54:19.933266: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 3.95295, expected 3.41014
2024-05-10 14:54:19.933273: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 3.54508, expected 3.00226
2024-05-10 14:54:19.933281: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 3.04094, expected 2.49812
2024-05-10 14:54:19.933288: E external/local_xla/xla/servi

227/227 ━━━━━━━━━━━━━━━━━━━━ 55s 171ms/step - accuracy: 0.5727 - loss: 22.9635 - val_accuracy: 0.6632 - val_loss: 0.6606
Epoch 2/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 18s 81ms/step - accuracy: 0.6627 - loss: 0.6452 - val_accuracy: 0.6836 - val_loss: 0.6126
Epoch 3/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 18s 81ms/step - accuracy: 0.6923 - loss: 0.5914 - val_accuracy: 0.6907 - val_loss: 0.5991
Epoch 4/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 18s 81ms/step - accuracy: 0.6891 - loss: 0.5831 - val_accuracy: 0.7062 - val_loss: 0.5956
Epoch 5/5
227/227 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.7232 - loss: 0.5368 - val_accuracy: 0.6830 - val_loss: 0.6000


In [5]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

121/122 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6736 - loss: 0.6009

2024-05-10 14:55:53.749876: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.02106, expected 2.15839
2024-05-10 14:55:53.749937: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.93624, expected 4.07356
2024-05-10 14:55:53.749959: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 5.5711, expected 4.70842
2024-05-10 14:55:53.749967: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 4.78519, expected 3.92251
2024-05-10 14:55:53.749975: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 5.9328, expected 5.07012
2024-05-10 14:55:53.749983: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 4.35545, expected 3.49277
2024-05-10 14:55:53.749990: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.66165, expected 5.79898
2024-05-10 14:55:53.749998: E external/local_xla/xla/serv

122/122 ━━━━━━━━━━━━━━━━━━━━ 14s 119ms/step - accuracy: 0.6736 - loss: 0.6008
Test Accuracy: 0.670869767665863
122/122 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step


In [6]:
# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.6488423373759648
Recall: 0.9978804578211107
F1-score: 0.786370469350259
